##### Imports

In [1]:
import cv2 as cv
import tools
import random
from os.path import join
import degpackage
import enhpackage
import qualitypackage
import edgepackage

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##### Data Loader

In [2]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

rgb_dir = 'dataset/rgb'
boundary_dir = 'dataset/boundary'
batch_size = 100

rgb_names, boundary_names = tools.im_exists(rgb_dir, batch_size)

dataset_rgb = []
dataset_depth = []
square_distances = []
i_ys = []

for image in rgb_names:
  dataset_rgb.append(cv.imread(join('dataset/rgb', image)))
  dataset_depth.append(cv.imread(join('dataset/depth', image)))

for image in boundary_names:
  boundary = cv.imread(join('dataset/boundary', image))[:, :, 0].astype(np.uint8)
  distance = cv.distanceTransform(cv.bitwise_not(boundary), cv.DIST_L1, maskSize=3)
  square_distances.append(distance * distance)
  i_ys.append(np.sum(boundary) / 255)


##### Degradation and Enhancement

In [3]:
deg_paths = ['deg_haze', 'deg_haze_depth', 'deg_random_noise', 'deg_reduce_contr_bright', 'deg_none']
enh_paths = ['enh_none', 'enh_increase_contr_bright']
img_paths = []

for method_deg in deg_paths:
  degpackage.degrade(method_deg, dataset_rgb, dataset_depth, rgb_names)

for method_enh in enh_paths:
  for method_deg in deg_paths:
    enhpackage.enhance(method_enh, method_deg, dataset_depth, rgb_names)


##### Quality Evaluation

In [4]:
for deg in deg_paths:
  for enh in enh_paths:
    img_paths.append(join(deg, enh))

for path in img_paths:
  qualitypackage.eval_quality(dataset_rgb, path, rgb_names)

Data successfully writen to:  deg_haze\enh_none\_quality.csv
Data successfully writen to:  deg_haze\enh_increase_contr_bright\_quality.csv
Data successfully writen to:  deg_haze_depth\enh_none\_quality.csv
Data successfully writen to:  deg_haze_depth\enh_increase_contr_bright\_quality.csv


KeyboardInterrupt: 

##### Edge Detection

In [ ]:
for path in img_paths:

  modded_rgb = []

  for rgb in rgb_names:
    modded_rgb.append(cv.imread(join(path, rgb)))

  modded_boundary = edgepackage.edge_detection(path, modded_rgb, boundary_names)
  edgepackage.edge_evaluation(path, modded_boundary, boundary_names, square_distances, i_ys)